In [4]:
import fastsparsegams
import numpy as np
import copy

In [5]:
## Generate synthetic, binary data. 
# first 10 features are used, none others. Normal noise, features all random integers
# (documentation of this setup can be found at https://tnonet.github.io/L0Learn/tutorial.html
#  and this setup is also used in the tutorial example for fastsparsegams)
np.random.seed(4)  # fix the seed to get a reproducible result
n, p, k = 1000, 100, 10
X = np.random.randint(2, size=(n, p)).astype(float)
B = np.zeros(p)
B[: (k // 2)] = 1
B[(k // 2) : k] = -1
e = np.random.normal(size=(n,)) / 2
y = np.sign(X @ B + e)

In [6]:
#TODO: remove features, and introduce corresponding missingness, under MCAR or MAR
removal_prop = 0.1

In [7]:
#train/test split
train_prop = .9
Xtrain, ytrain = copy.copy(X[:int(train_prop*n)]), copy.copy(y[:int(train_prop*n)])
Xtest, ytest = copy.copy(X[int(train_prop*n):]), copy.copy(y[int(train_prop*n):])

In [8]:
fit_model_3 = fastsparsegams.fit(
    Xtrain, ytrain, loss="Exponential", max_support_size=20, algorithm="CDPSI"
)

In [9]:
### print out results (which are across many distinct L0 penalties)
fit_model_3

,l0,support_size,intercept,converged,l2
0,32286.533979,0,0.006667,True,0.0
1,31963.668639,0,0.006667,True,0.0
2,25570.934912,0,0.006667,True,0.0
3,20456.747929,0,0.006667,True,0.0
4,16365.398343,0,0.006667,True,0.0
5,13092.318675,0,0.006667,True,0.0
6,10473.854940,0,0.006667,True,0.0
7,8379.083952,0,0.006667,True,0.0
8,6703.267161,0,0.006667,True,0.0
9,5362.613729,0,0.006667,True,0.0


In [12]:
# Take one model which narrows down to just 10 features, the 40th one.
# Demonstrate these are the ground truth 10 features, plus an intercept term. 
lambda0 = fit_model_3.support_size[0][40]
print(np.where(fit_model_3.coeff(support_size=lambda0).toarray() > 1e-8)[0])
print(np.where(np.abs(fit_model_3.coeff(lambda_0=lambda0).toarray()) > 1e-8)[0])
# TODO: formal procedure for selecting which lambda to use? 

TypeError: FitModel.coeff() got an unexpected keyword argument 'support_size'

In [14]:
yhat_train = np.where(fit_model_3.predict(Xtrain, support_size=1).flatten() > 0.5, 1, -1)
yhat_test = np.where(fit_model_3.predict(Xtest, lambda_0=lambda0).flatten() > 0.5, 1, -1)

TypeError: FitModel.predict() got an unexpected keyword argument 'support_size'

In [ ]:
print(np.mean(yhat_train == ytrain))
print(np.mean(yhat_test == ytest))


In [ ]:
#ground truth test accuracy: 
np.mean(np.sign(X @ B)[int(n*train_prop):] == ytest )
#It seems strange for the model's accuracy to exceed this. 
#    (TODO: the model's accuracy seems to exceed this fairly often. Investigate why.)

In [3]:
import numpy as np
a = np.ones(3)
b = np.ones(3)
b[1] = -1
b[2] = 0
np.mean(a == b)

0.3333333333333333